In [58]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [59]:
df = pd.read_csv('price_data.csv')

In [60]:
df.drop(['fuelAmount','totalConcession', 'tatkalFare','availability','reservationCharge','serviceTax','cateringCharge','timeStamp','dynamicFare','otherCharge','baseFare','duration'],axis='columns',inplace=True)
#first 3 are all zero 
#availability is json
#reservationCharge,serviceTax is based on classCode for avoiding Multicollinearity  
#'baseFare','duration' ----------Multicollinearity
#cateringCharge,dynamicFare,'otherCharge' 99% of values are zero
# timeStamp not needed

In [61]:
df.superfastCharge = df.superfastCharge.apply(lambda x:0 if x==0 else 1)

In [62]:
df = df.rename(columns={'superfastCharge':'SuperFast'})

In [63]:
df.drop_duplicates(subset=['SuperFast', 'trainNumber', 'fromStnCode', 'toStnCode',
       'classCode', 'distance'],ignore_index=True,keep='last',inplace=True)

In [64]:
#df.trainNumber = df.trainNumber.astype(str)

In [65]:
df = df[['trainNumber', 'fromStnCode', 'toStnCode',
       'classCode', 'distance','SuperFast', 'totalFare']]

In [66]:
X = df.drop(['totalFare'],axis='columns')
y = df.totalFare

In [67]:
cat_columns = [x for x in X.columns if X[x].dtype=='O']
num_columns = [x for x in X.columns if X[x].dtype!='O']


In [68]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [69]:

cat_pipeline = Pipeline(
steps=[
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('cat_pipeline',cat_pipeline,cat_columns)
],remainder='passthrough')

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7,random_state=45)

In [80]:
X_train.shape,y_train.shape

((208446, 6), (208446,))

In [81]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [82]:
X_train.shape

(208446, 3058)

In [83]:
from sklearn.linear_model import LinearRegression

In [84]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [85]:
model = RandomForestRegressor()
model.fit(X_train,y_train)

RandomForestRegressor()

In [86]:
model.score(X_train,y_train)

0.9991794095952091

In [87]:
model.score(X_test,y_test)

0.9937668732785585

In [19]:
#0.866980573558711
#0.8633145591672289

In [88]:
price = pd.read_csv('price_data.csv')

In [89]:
schedule = pd.read_csv('schedules.csv')

In [90]:
schedule.head()

,trainNumber,trainName,stationFrom,stationTo,trainRunsOnMon,trainRunsOnTue,trainRunsOnWed,trainRunsOnThu,trainRunsOnFri,trainRunsOnSat,trainRunsOnSun,timeStamp,stationList
0,961,VALLEY QUEEN SPL,MJ,MJMG,Y,Y,N,Y,Y,N,N,2023-10-15T13:04:05.392,"[{'stationCode': 'MJ', 'stationName': 'MARWAR ..."
1,1027,DR GKP SPECIAL,DR,GKP,N,Y,N,Y,N,Y,Y,2023-10-15T13:04:36.625,"[{'stationCode': 'DR', 'stationName': 'DADAR',..."
2,1065,DADAR DHULE SPL,DR,DHI,Y,N,N,N,Y,N,Y,2023-10-15T13:05:16.833,"[{'stationCode': 'DR', 'stationName': 'DADAR',..."
3,1066,DHULE DADAR SPL,DHI,DR,Y,Y,N,N,N,Y,N,2023-10-15T13:05:17.545,"[{'stationCode': 'DHI', 'stationName': 'DHULE'..."
4,1127,LTT BPQ SPL,LTT,BPQ,N,Y,N,N,N,N,N,2023-10-15T13:05:44.865,"[{'stationCode': 'LTT', 'stationName': 'LOKMAN..."


In [ ]:
import json

In [ ]:
#get_availability()
y = df.availability.tolist()[58]
parse_data = json.loads(y.replace("'", "\""))
parse_data_df = pd.DataFrame(parse_data)
parse_data_df


In [ ]:
#get_schedule_by_train_no
x = df1.stationList.tolist()[97]
parsed_data = json.loads(x.replace("'", "\""))
parsed_data_df = pd.DataFrame(parsed_data)
parsed_data_df.drop(['routeNumber','stnSerialNumber','boardingDisabled'],axis='columns',inplace=True)
parsed_data_df